In [2]:
pip install pdfplumber pandas openpyxl pymupdf

In [3]:
import fitz  # Import PyMuPDF for PDF manipulation
import pdfplumber  # Import pdfplumber for text extraction from PDFs
import pandas as pd  # Import pandas for data manipulation and analysis
import re  # Import regular expressions for pattern matching

def extract_voucher_details(text):
    """
    Extracts voucher details from the provided text.

    Args:
        text (str): The text content of a PDF page.

    Returns:
        dict: A dictionary containing the voucher name, number, and date.
    """

    # Extract Voucher Name using regex
    voucher_name_match = re.search(r"(\w*\s*Voucher)", text)
    voucher_name = voucher_name_match.group(1) if voucher_name_match else "N/A"

    # Extract Voucher Number
    no_match = re.search(r"No\.\s*:\s*(\S+)", text)
    no = no_match.group(1) if no_match else "N/A"

    # Extract Date using various date formats without relying on 'Dated'
    date_match = re.search(r"\b(\d{1,2}-\w{3}-\d{2,4}|\d{1,2}/\d{1,2}/\d{2,4}|\d{1,2}-\d{1,2}-\d{2,4})\b", text)
    date = date_match.group(1).strip() if date_match else "N/A"

    return {
        "Voucher Name": voucher_name,
        "No": no,
        "Date": date,
    }

# Define the path to the PDF file
pdf_path = "data.pdf"

# Initialize lists to store extracted data
extracted_particulars = []  # For storing particulars
voucher_data = []  # For storing voucher details
extracted_onaccountof = []  # For storing 'On Account of' data

# Process the PDF using pdfplumber
with pdfplumber.open(pdf_path) as pdf:
    # Iterate through each page of the PDF
    for page_num, page in enumerate(pdf.pages):
        text = page.extract_text()  # Extract text from the page
        if "Voucher" in text:  # Process only pages that contain 'Voucher'
            details = extract_voucher_details(text)  # Extract voucher details
            voucher_data.append(details)  # Add extracted data to the list

# Extract particulars using PyMuPDF
with fitz.open(pdf_path) as pdf:
    # Iterate through each page of the PDF
    for page_num in range(pdf.page_count):
        page = pdf[page_num]
        text = page.get_text()  # Get text content of the page

        if "Voucher" in text:  # Process only pages that contain 'Voucher'
            lines = text.split('\n')  # Split text into lines

            # Extract particulars that follow 'Particulars'
            for i, line in enumerate(lines):
                if "Particulars" in line:
                    for j in range(i + 1, len(lines)):
                        first_line_below = lines[j].strip()  # Get the next line
                        # Validate the line to exclude certain keywords
                        if first_line_below and "|" not in first_line_below and "Amount" not in first_line_below and "Debit" not in first_line_below and "Credit" not in first_line_below and "Account:" not in first_line_below:
                            extracted_particulars.append(first_line_below)  # Append valid line to extracted particulars
                            break
                    break  # Stop after processing the particulars

            # Locate "On Account of" and collect relevant lines below it
            for i, line in enumerate(lines):
                if "On Account of" in line:
                    current_extracted = []  # List to hold 'On Account of' lines
                    # Collect all subsequent lines until a keyword is encountered
                    for j in range(i + 1, len(lines)):
                        first_line_below = lines[j].strip()
                        # Check if the line is valid to be included
                        if first_line_below:
                            # Break if encountering specific keywords
                            if (any(keyword in first_line_below for keyword in ["Amount (in words)", "Debit", "Dated", "Credit", "Amount", "Dr", "Amount (in words) :", "Amount {in words) :"])):
                                break  # Stop processing when encountering any of these
                            current_extracted.append(first_line_below)  # Append valid line
                        else:
                            break  # Stop if an empty line is encountered
                    text = " ".join(current_extracted)  # Join collected lines into a single string
                    extracted_onaccountof.append(text)  # Append to extracted 'On Account of' data
                    break  # Stop searching once processed

# Create DataFrames for each type of extracted data
df_vouchers = pd.DataFrame(voucher_data)  # DataFrame for voucher details
df_particulars = pd.DataFrame(extracted_particulars, columns=['Particulars'])  # DataFrame for particulars
df_accountof = pd.DataFrame(extracted_onaccountof, columns=['On account of'])  # DataFrame for 'On Account of' data

# Combine DataFrames into a single DataFrame
df_combined = pd.concat([df_vouchers, df_particulars, df_accountof], axis=1)

# Save the combined DataFrame to a single Excel file
df_combined.to_excel("extracted_data.xlsx", index=False)

print("Data extracted and saved to 'extracted_data.xlsx'")

Data extracted and saved to 'extracted_data.xlsx'
